Prompts
A prompt for a language model is a set of instruction or input provided by a user to guid the model's response.
Prompts can be defined with **PromptTemplate** and **ChatPromptTemplate**

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Tell me a {adjective} joke about {topic}?")
result = prompt_template.format(adjective="funny", topic="chicken")
result

'Human: Tell me a funny joke about chicken?'

With **ChatPromptTemplate**, it is required to provide an additional parameter called *role*. For example, OpanAi Chat Completion API have System, AI, or Human role.

In [19]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Your name is {name}"),
        ("human", "Hello, how are you?"),
        ("assistant", "I am fine, thank you!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Alice", user_input="What is the weather like today?")

messages

[SystemMessage(content='You are a helpful assistant. Your name is Alice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am fine, thank you!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the weather like today?', additional_kwargs={}, response_metadata={})]

**Chat Models**

In [20]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")

llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_version=AZURE_OPENAI_API_VERSION,
    api_key=AZURE_OPENAI_KEY
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?")
]

response = llm.invoke(messages)
response

AIMessage(content='The capital of France is **Paris**.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CknPe3b8yFyjo1wQe0fBffKfto8OK', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'sa

**Completion**

In [21]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)

response = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure services support this too?"},
    ]
)

response.choices[0].message.content

"Yes, many other Azure services support **Customer Managed Keys (CMK)**, allowing organizations to have greater control over their data encryption keys, ensuring compliance with security and regulatory requirements. With CMK, you can use encryption keys that you control and manage through **Azure Key Vault** or another key management solution. Here's a list of Azure services that support CMK, as of now:\n\n### **Azure Services Supporting Customer Managed Keys**\n1. **Azure Storage Account** \n   - Blob storage, file storage, queue storage, and table storage support CMK for encrypting data at rest.\n\n2. **Azure Data Lake Storage**\n   - Both Gen1 and Gen2 support CMK using Azure Key Vault.\n\n3. **Azure SQL Database, Azure SQL Managed Instance, and Azure Synapse Analytics**\n   - Transparent Data Encryption (TDE) supports CMK instead of service-managed keys.\n\n4. **Azure Cosmos DB**\n   - Offers encryption at rest with support for customer-managed keys.\n\n5. **Azure Disk Encryption**

In [22]:
response.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Yes, many other Azure services support **Customer Managed Keys (CMK)**, allowing organizations to have greater control over their data encryption keys, ensuring compliance with security and regulatory requirements. With CMK, you can use encryption keys that you control and manage through **Azure Key Vault** or another key management solution. Here's a list of Azure services that support CMK, as of now:\n\n### **Azure Services Supporting Customer Managed Keys**\n1. **Azure Storage Account** \n   - Blob storage, file storage, queue storage, and table storage support CMK for encrypting data at rest.\n\n2. **Azure Data Lake Storage**\n   - Both Gen1 and Gen2 support CMK using Azure Key Vault.\n\n3. **Azure SQL Database, Azure SQL Managed Instance, and Azure Synapse Analytics**\n   - Transparent Data Encryption (TDE) supports CMK instead of service-managed keys.\n\n4. **Azure Cosmos DB**\n   - Offer

**Output Parsers**
- Language models output text but many times we may want to get more structured information thant just text
- An output parser must implement 2 methods:
  - *Get format instructions*: A method which returns a string containing instructions for how the output of a language model should be formatted
  - *Parse*: A method which takes in a string(assumed to be the response from a language model) and parses it into some structure
- LangChain has several built-int output parsers:
  - List parser
  - Datetime parser
  - Enum parser
  - ConvoOutputParser

**Chains**
- Using an LLM in a single step is fine for simple applications but more complex applications require chaining LLMs - either with each other LLM or with other components.
- There are several chain types:
  - LLM chain: Is the most common type of chaining in LLM application
  - Router chain: Route to different operations based on user input
  - Sequential chain: Combine multiple LLM chain into a pipeline
  - Transformation chain: Transform inputs with LLM

In [23]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_version=AZURE_OPENAI_API_VERSION,
    api_key=AZURE_OPENAI_KEY,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Use LLM Chain
prompt_template = """Tell me a joke about {topic}?
Answer:
"""

llm_chain = ChatPromptTemplate.from_template(prompt_template) | llm
result = llm_chain.invoke({"topic": "robot"})
result.content

"Sure! Here's a robot joke for you:\n\nWhy did the robot go on a diet?  \nBecause he had a byte problem! 🤖"